# Loading Experiment Data

In this notebook, we start by loading the data collected while running different experiment-wares (in this case, SAT solvers), and perform some preprocessing on this data to allow its use for further analysis in dedicated notebooks.

## Imports

We first need to import the modules we need to load the data.
In particular, we must obviously import *Metrics-Wallet*, which we will use to deal with our data.

In [1]:
from itertools import product
from metrics.wallet import BasicAnalysis

## Reading the data

The next step is to read the data from the CSV files storing the runtime of our different solvers.
This data is described in the file [`scalpel_config.yml`](config/scalpel_config.yml), and automatically parsed by *Metrics-Scalpel* to create a `BasicAnalysis` object.

In [2]:
analysis = BasicAnalysis(input_file='config/scalpel_config.yml', log_level='WARNING')

The `BasicAnalysis` object instantiated above provides elementary and general methods for preprocessing our data before actually analyzing the results (which will require more specific methods as it can be seen in the dedicated notebooks).

An important thing to do now is to visualize the collected data, to make sure that everything was properly read.
This can be achieved by looking at the data-frame that has been built inside the `BasicAnalysis` object.

In [3]:
analysis.data_frame

input experiment_ware  \
0    /home/evaluation/evaluation/pub/bench/sc2020/m...         glucose   
400  /home/evaluation/evaluation/pub/bench/sc2020/m...          kissat   
1    /home/evaluation/evaluation/pub/bench/sc2020/m...         glucose   
401  /home/evaluation/evaluation/pub/bench/sc2020/m...          kissat   
2    /home/evaluation/evaluation/pub/bench/sc2020/m...         glucose   
..                                                 ...             ...   
798  /home/evaluation/evaluation/pub/bench/sc2020/m...          kissat   
398  /home/evaluation/evaluation/pub/bench/sc2020/m...         glucose   
799  /home/evaluation/evaluation/pub/bench/sc2020/m...          kissat   
399  /home/evaluation/evaluation/pub/bench/sc2020/m...         glucose   
525  /home/evaluation/evaluation/pub/bench/sc2020/m...          kissat   

     cpu_time decision  timeout  success  user_success  missing  \
0       342.0      SAT  10000.0     True          True    False   
400     318.0      SAT  10000.0     True          True    False   
1     10000.0  UNKNOWN  10000.0     True          True    False   
401    2224.0      SAT  10000.0     True          True    False   
2     10000.0  UNKNOWN  10000.0     True          True    False   
..        ...      ...      ...      ...           ...      ...   
798     562.0    UNSAT  10000.0     True          True    False   
398    2488.0    UNSAT  10000.0     True          True    False   
799     197.0    UNSAT  10000.0     True          True    False   
399   10000.0  UNKNOWN  10000.0     True          True    False   
525      65.0      SAT  10000.0     True          True    False   

     consistent_xp  consistent_input  error  
0             True              True  False  
400           True              True  False  
1             True              True  False  
401           True              True  False  
2             True              True  False  
..             ...               ...    ...  
798           True              True  False  
398           True              True  False  
799           True              True  False  
399           True              True  False  
525           True              True  False  

[800 rows x 11 columns]

## Checking the success and consistency of the results

During our analysis, we will need to know whether a given experiment was successful.

In [4]:
def is_success(xp):
    """
    This function checks that a solver either found a solution, or proved
    the input to be unsatisfiable.
    """
    return xp['decision'] == 'SAT' or xp['decision'] == 'UNSAT'

To make sure that our experiments are consistent, we also need to compare the results obtained by the different experiment-wares.

In [5]:
def is_consistent_by_input(df_input):
    """
    This function checks that, for a given input, all decisions made by the
    different solvers are consistent.
    In particular, a solver cannot answer UNSAT if another solver has found
    a solution.
    """
    decisions = df_input['decision'].unique()
    return 'SAT' not in decisions or 'UNSAT' not in decisions

We can now use the functions above to check the consistency of the different experiments in the analysis.

In [6]:
analysis.check_success(is_success)
analysis.check_input_consistency(is_consistent_by_input)

No warning has been raised, so all results are consistent.
This can also be seen in the following table, as it does not contain any experiment.

In [7]:
analysis.error_table()

Empty DataFrame
Columns: [input, experiment_ware, cpu_time, decision, timeout, success, user_success, missing, consistent_xp, consistent_input, error]
Index: []

## Summary and export of the analysis

We can now give a summary of the analysis, that we obtain through the following table.

In [8]:
analysis.description_table()

analysis
n_experiment_wares                                                          2
n_inputs                                                                  400
n_experiments                                                             800
n_missing_xp                                                                0
n_inconsistent_xp                                                           0
n_inconsistent_xp_due_to_input                                              0
more_info_about_variables       <analysis>.data_frame.describe(include='all')

Finally, the analysis is exported, both to share the data to allow the reproducibility of the analysis, and to reuse it in other notebooks dedicated to more specific analyses.

In [9]:
analysis.export('.cache')